In [1]:
import pandas as pd

import general_utils
%load_ext autoreload
%autoreload 2
%aimport algorithm_utils

In [2]:
from algorithm_utils import *
import general_utils as gu

In [3]:
timeframe = "15m"
higher_timeframe = general_utils.find_higher_timeframe(timeframe)

In [69]:
testing_length = 500
start_index = -8 * testing_length - 1
pair_name = "SOLUSDT"

original_pair_df: pd.DataFrame = gu.load_local_data(pair_name=pair_name, timeframe=timeframe)

if testing_length > 0:
    pair_df: pd.DataFrame = original_pair_df.iloc[start_index:start_index + testing_length].reset_index(drop=True)
else:
    pair_df: pd.DataFrame = original_pair_df

# Higher timeframe data for determining starting point
htf_pair_df: pd.DataFrame = gu.load_higher_tf_data(pair_name=pair_name, timeframe=higher_timeframe)

print("LTF length is", len(pair_df))
print("HTF length is", len(htf_pair_df))

LTF length is 500
HTF length is 5845


In [70]:
gu.reset_logs()

print("Original pair_df starting point is at time", pair_df.iloc[0].time)

algo = Algo(pair_df, pair_name, allowed_verbosity=3)
initial_data_start_time = algo.pair_df.iloc[0].time

corrected_pair_df = create_filtered_pair_df_with_corrected_starting_point(htf_pair_df, initial_data_start_time,
                                                                          original_pair_df, timeframe, higher_timeframe)

print("Updated pair_df starting point to", corrected_pair_df.iloc[0].time)

algo = Algo(corrected_pair_df, "BTCUSDT", allowed_verbosity=1)

algo.init_zigzag(last_pivot_type="valley", last_pivot_candle_pdi=0)
h_o_starting_point: int = algo.zigzag_df.iloc[0].pdi

algo.calc_h_o_zigzag(h_o_starting_point)

# corrected_pair_df = pair_df.copy()

# print("Updated pair_df starting point to", corrected_pair_df.iloc[0].time)

# algo = Algo(corrected_pair_df, pair_name, allowed_verbosity=3)

# algo.init_zigzag(last_pivot_type="peak", last_pivot_candle_pdi=3)

# h_o_starting_point: int = algo.zigzag_df.iloc[0].pdi

# algo.calc_h_o_zigzag(h_o_starting_point)


# algo.h_o_indices.append(87)
# algo.h_o_indices.append(315)
# algo.h_o_indices.append(431)
# print(algo.find_lpls())
# h_o_zigzag_df = generate_h_o_zigzag(zigzag_df)
# pbos_df = h_o_zigzag_df.loc[h_o_zigzag_df.is_pbos]
# bos_df = find_confirmed_boss(pbos_df, pair_df)
# lpl_df = find_lpls(bos_df, zigzag_df)
# lplb_df = find_lplbs(bos_df, zigzag_df)

# boxes_list = []
# for lpl in lpl_df.itertuples():
#     start_index = lpl.pair_df_index
#     base_candle = Candle.create(pair_df.iloc[lpl.pair_df_index])
#     box_type = "long" if lpl.pivot_type == "valley" else "short"
#     box_to_add = Box(base_candle, b|ox_type)
#     box_to_add.check_box_ent

#     ries(pair_df)
#     boxes_list.append(box_to_add)


Original pair_df starting point is at time 2024-07-21 08:00:00
Updated pair_df starting point to 2024-07-14 14:30:00
('Added starting point', np.int64(0))
('',)
('Added BOS', 68)
('',)
('BOS #', 68, 'break at', np.int64(104))
('Added extremum of type', 'lowest low', 'at', np.int64(82))
('Setting pattern start to', np.int64(95))
('',)
('Added BOS', 115)
('',)
('BOS #', 115, 'break at', np.int64(122))
('Added extremum of type', 'lowest low', 'at', np.int64(116))
('Setting pattern start to', np.int64(121))
('',)
('Added BOS', 134)
('CHOCH #', 116, 'break at', np.int64(161))
('Setting pattern start to', np.int64(159))
('',)
('Added BOS', 168)
('CHOCH #', 134, 'break at', np.int64(197))
('Setting pattern start to', np.int64(193))
('',)
('Added BOS', 235)
('BOS #', 235, 'break at', np.int64(446))
('Added extremum of type', 'lowest low', 'at', np.int64(296))
('Setting pattern start to', np.int64(442))
('',)
('Added BOS', 447)
('',)
('BOS #', 447, 'break at', np.int64(477))
('Added extremum of

In [71]:
pt = gu.PlottingTool(width=1400, height=800)
pt.draw_candlesticks(corrected_pair_df)
pt.draw_zigzag(algo.zigzag_df)

pt.draw_points_with_label(algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pdi,
                          algo.zigzag_df[algo.zigzag_df.pdi.isin(algo.h_o_indices)].pivot_value,
                          draw_line=True)

for segment in algo.segments:
    print(segment)
    segment.filter_candlestick_range(algo)
    segment.find_order_blocks(algo)
    pt.draw_segment_bbox(segment)
    for ob in segment.ob_list:
        position_entry_pdi = ob.position.find_entry_within_segment(segment)
        if position_entry_pdi is not None:
            pt.draw_box(ob, len(algo.pair_df))
            ob.position.enter(position_entry_pdi)

    for ob in segment.ob_list:
        if ob.position.status != "ENTERED":
            continue

        exit_check_candles: pd.DataFrame = algo.pair_df.iloc[ob.position.entry_pdi:]
        candle_sentiments: pd.Series = exit_check_candles.apply(lambda candle: ob.position.detect_candle_sentiment(candle), axis=1)

        # Find the first instance where a STOPLOSS or FULL_TARGET happens in the candle_sentiments Series.
        mask = candle_sentiments.apply(lambda row: row[0] == "FULL_TARGET" or row[0] == "STOPLOSS")
        exit_index = mask.idxmax() if mask.any() else None

        if exit_index is not None:
            exit_code = candle_sentiments.loc[exit_index][0]

            # If the first exit event to happen is a full target, that means we have the maximum profit, and we exit the position.
            if exit_code == "FULL_TARGET":
                # This snippet is used to register each target individually, before the full target event happens. This is used for validating target
                # hits and troubleshoot bugs later using the ob.position.target_hit_pdis property which stores the PDI's of the candles that hit 
                # the targets.
                targets_before_full_target_cum_max: pd.Series = candle_sentiments.loc[:exit_index].apply(
                    lambda row: row[1] if row[0] == "TARGET" else 0).cummax()
                max_target_changes: pd.Series = targets_before_full_target_cum_max[targets_before_full_target_cum_max.diff() > 0]

                # Register the targets before the final full target
                for target_id, target_hit_pdi in zip(max_target_changes.values, max_target_changes.index):
                    ob.position.register_target(target_id, target_hit_pdi)

                # Register the final, full target
                ob.position.register_target(len(ob.position.target_list), exit_index)

                # Exit the position
                ob.position.exit(exit_code="FULL_TARGET", exit_pdi=exit_index)


            # If the first exit event to happen is a stoploss, we must check what the highest target reached was.
            elif exit_code == "STOPLOSS":
                # This snippet is used to register each target individually, before the stoploss event happens.
                targets_before_stoploss_cum_max: pd.Series = candle_sentiments.loc[:exit_index].apply(
                    lambda row: row[1] if row[0] == "TARGET" else 0).cummax()
                max_target_changes: pd.Series = targets_before_stoploss_cum_max[targets_before_stoploss_cum_max.diff() > 0]

                # Register the targets before the final full target
                for target_id, target_hit_pdi in zip(max_target_changes.values, max_target_changes.index):
                    ob.position.register_target(target_id, target_hit_pdi)

                ob.position.exit(exit_code="STOPLOSS", exit_pdi=exit_index)

        # print(ob) 
        # print("TARGETS", ob.position.target_list)
        # print("STOPLOSS", ob.position.stoploss)
        # print("FIRST EXIT INDEX", exit_index)   
        # print(candle_sentiments.iloc[exit_index])

Ascending segment starting at 0 ending at 103 OB formation at 83
Ascending segment starting at 82 ending at 121 OB formation at 117
Ascending segment starting at 168 ending at 445 OB formation at 243
Ascending segment starting at 296 ending at 476 OB formation at 462
Ascending segment starting at 461 ending at 588 OB formation at 572
Ascending segment starting at 571 ending at 675 OB formation at 635
Ascending segment starting at 1054 ending at 1391 OB formation at 1254
Descending segment starting at 1402 ending at 1656 OB formation at 1562
Descending segment starting at 1613 ending at 1728 OB formation at 1720
Descending segment starting at 1722 ending at 1824 OB formation at 1755
Descending segment starting at 1767 ending at 1923 OB formation at 1847
Descending segment starting at 1853 ending at 2057 OB formation at 2039
Ascending segment starting at 2079 ending at 2254 OB formation at 2245
Ascending segment starting at 2250 ending at 2391 OB formation at 2276
Descending segment star

In [72]:
pt.show()

In [73]:
import pandas as pd
from openpyxl import load_workbook

def generate_positions_excel(output_file):
    # List to store position data
    positions_data = []

    # Iterate through each segment in the Algo object
    for segment in algo.segments:
        # Iterate through each order block in the segment
        for ob in segment.ob_list:
            # Extract the position information
            position = ob.position
            if len(position.target_hit_pdis) == 1:
                target_hit_times_list = [algo.convert_pdis_to_times(position.target_hit_pdis)] 
                
            else:
                target_hit_times_list = algo.convert_pdis_to_times(position.target_hit_pdis)
                
            position_data = {
                'Position ID': position.parent_ob.id,
                'Status': position.status,
                'Entry time': algo.convert_pdis_to_times(position.entry_pdi),
                'Exit time': algo.convert_pdis_to_times(position.exit_pdi),
                'Target hit times': [time.strftime("%Y-%m-%d %X") for time in target_hit_times_list],
                'Type': position.type,
                'Entry price': position.entry_price,
                'Stoploss': position.stoploss,
                'Target list': [round(float(target), 6) for target in position.target_list]
            }
            # Append the position data to the list
            positions_data.append(position_data)

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(positions_data)

    # Write the DataFrame to an Excel file
    df.to_excel(output_file, index=False)

    # Adjust column widths
    adjust_column_widths(output_file)

def adjust_column_widths(output_file):
    # Load the workbook and select the active worksheet
    wb = load_workbook(output_file)
    ws = wb.active

    # Adjust column widths
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column name
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width

    # Save the workbook
    wb.save(output_file)

# Example usage
# Assuming algo is an instance of the Algo class
generate_positions_excel('positions.xlsx')

In [84]:
import pandas as pd
from lightweight_charts.widgets import StreamlitChart

chart = StreamlitChart(width=900, height=600)

chart.set(algo.pair_df)

chart.load()

ModuleNotFoundError: streamlit.components.v1.html was not found, and must be installed to use StreamlitChart.